In [ ]:
from CWRU_main import *
from CWRU_evaluation import *
from CWRU_preprocessing import *

In [ ]:
HP = "0"

N_ITER = 30
CV = 5
TEST_SIZE = 0.2

N_REPEATS = 5

In [ ]:
# Data preprocessing steps

from CWRU_preprocessing import *
from sklearn.model_selection import train_test_split

path = "CWRU/" + HP + "HP/"
df = load_mat_time_signals_to_df(path)
data = preprocess_time_pipeline(df)

print(data["X_train_flat"].shape)
print(data["X_train_cnn"].shape)
print(data["y_train_cat"].shape)
print("Classes:", list(data["label_encoder"].classes_))

In [ ]:
X_train = data["X_train_flat"]
y_train = data["y_train_encoded"]
X_test = data["X_test_flat"]
y_test = data["y_test_encoded"]

X_train_cnn = data["X_train_cnn"]
y_train_cat = data["y_train_cat"]
X_test_cnn = data["X_test_cnn"]
y_test_cat = data["y_test_cat"]
le = data["label_encoder"]

In [ ]:
plot_tsne(
    X=X_train,  # FFT feature space
    y_encoded=y_train,
    label_encoder=le,
    title="t-SNE in Time-Domain",
    save_path="plots/0HP_tsne_fft_6classes.png"
)

In [ ]:
# X_train = X_train_flat
# y_train = y_train_encoded
# X_test = X_test_flat
# y_test = y_test_encoded

In [ ]:
# rf_search = load_tuned_model(model_name="rf", path="search_results/" + HP + "HP/search_rf.pkl")
rf_search = load_tuned_model(model_name="rf", path="search_results/" + HP + "HP/search_rf.pkl")
xgb_search = load_tuned_model(model_name="xgb", path="search_results/" + HP + "HP/search_xgb.pkl")
mlp_search = load_tuned_model(model_name="mlp", path="search_results/" + HP + "HP/search_mlp.pkl")
svm_search = load_tuned_model(model_name="svm", path="search_results/" + HP + "HP/search_svm.pkl")

In [ ]:
rf_best_params = rf_search.best_params_
xgb_best_params = xgb_search.best_params_
mlp_best_params = mlp_search.best_params_
svm_best_params = svm_search.best_params_

In [ ]:
print(rf_best_params)
print(xgb_best_params)
print(mlp_best_params)
print(svm_best_params)

In [ ]:
# # Step 2: Train and Evaluate using Best Params
# rf_preds = train_and_evaluate_rf(X_train, y_train, X_test, y_test, rf_params=rf_best_params, n_repeats=N_REPEATS)
# xgb_preds = train_and_evaluate_xgb(X_train, y_train, X_test, y_test, xgb_params=xgb_best_params, n_repeats=N_REPEATS)
# mlp_preds = train_and_evaluate_mlp(X_train, y_train, X_test, y_test, mlp_params=mlp_best_params, n_repeats=N_REPEATS)
# svm_preds = train_and_evaluate_svm(X_train, y_train, X_test, y_test, svm_params=svm_best_params, n_repeats=N_REPEATS)

In [ ]:
# One-hot encode after split
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

best_cnn_hp = load_best_hyperparameters("tuner_results/" + HP + "HP/cnn_tuning", "cnn_fault_classification")
print("\n🚀 Training and evaluating CNN model with best hyperparameters...")
cnn_preds = train_and_evaluate_cnn(X_train_cnn, y_train_cat, X_test_cnn, y_test_cat, best_cnn_hp, N_REPEATS)

In [ ]:
predictions_dict={
    "RF": rf_preds,
    "XGB": xgb_preds,
    "MLP": mlp_preds,
    "SVM": svm_preds,
    "CNN": cnn_preds
}

plot_prediction_accuracies(
    predictions_dict=predictions_dict,
    y_true=y_test,
    save_path="plots",
    filename=HP + "HP_model_accuracy_comp.png"
)

In [ ]:
best_model_name, best_pred = get_best_model_and_prediction(predictions_dict, y_test)

In [ ]:
label_map = {i: label for i, label in enumerate(le.classes_)}

# Best prediction from a model (e.g., RF)
best_rf_pred = svm_preds[np.argmax([accuracy_score(y_test, p) for p in rf_preds])]

plot_confusion_matrix_for_model(
    y_true=y_test,
    y_pred=best_pred,
    model_name=best_model_name,
    hp=HP+ "HP",
    save_path="plots",
    label_map=label_map,
    filename=HP + "HP_confusion_matrix.png"
)

In [ ]:
print(classification_report(y_test, best_rf_pred))